# Assignment 3 - more pitching#
#### CSCI 4831 - Sabermetrics ####
#### Instructor: Hoenigman ####
#### Due: Wednesday, March 6 by 3pm ####

**Use any of the datasets we've learned about so far to complete this assignment.** 

**Submit your notebook to the Assignment 3 link on Canvas. Include all csv files that you generate with your submission.**

## Pitching ###
In Assignment 2, you evaluated 5 Rockies pitchers by comparing pitch selection and movement of their pitches at home and away. In this next assignment, you will also evaluate pitchers in different environments or at different times. The specifics of how you do that are up to you. Your assignment needs to fulfill the following high-level objectives.

### 1. Pitchers to compare ###
(10 pts) You need pitchers that you will compare. Examples include groups of pitchers compared to other groups, an individual pitcher compared to another individual, or one pitcher compared to himself at different times in his career. 

### 2. Reason for your comparison ###
(10 pts) You need to have a reason for why you think the comparison is meaningful. For example, you might compare the star pitcher from a hitters park to the star pitcher at a pitchers park. This comparison could be meaningful to evaluate the effect of the run-scoring environment on their approach to pitching. A comparison that wouldn't be meaningful is average pitcher A compared to average pitcher B, both on average teams. That's not a result that anyone would care about. 

### 3. Basis for comparison ###
(30 pts) You need to describe how the pitchers will be compared and then implement your comparison. Examples of comparison include pitch movement and selection, BABIP, BABIP on specific pitches, OBP in different environments. Choose at least three criteria for pitcher comparison. 

### 4. Data for comparison ###
(10 pts) Focus on starting pitchers only because you will have more data. Use 3-5 years of data for each pitcher or time period. A pitcher will need to have at least 3 years of data to be used in this assignment.

### 5. Analysis ###
(20 pts) Your comparison needs to include a basic statistical analysis, such as t-tests, z-tests, and/or confidence intervals. You also need at least one visual presentation of your data, such as a histogram, bar chart, or line graph. Your visual should be appropriate for the type of data being displayed.

### 6. Presentation ###
(20 pts) You need a 2-page writeup explaining what you did and your results. Your presentation should have clearly marked sections for 

* Introduction
* Problem Definition
* Data
* Results

### Other notes ###
* Proofread your writeup for typos and proper grammar. 
* If you compare pitch movement, use the statcast fields for vx0, vz0, ax, az, plate_x, plate_z variables instead of pfx_x, pfx_z, I'm not sure what those do.
* Include all code and the writeup together in one notebook.

In [78]:
#from pybaseball import statcast

#pitches = statcast('2016-01-01', '2018-12-20')

In [79]:
#pitches_2016 = statcast('2016-04-02', '2016-11-02')

In [80]:
import pandas as pd
#pitches_2016_2018 = pd.concat([pitches, pitches_2016], ignore_index=True).drop("index", axis=1)

In [81]:
#pitches_2016_2018.to_csv("pitches_2016_2018.csv")

In [19]:
from pybaseball.lahman import pitching
from pybaseball.lahman import download_lahman

download_lahman()

p = pitching()
p_2016_2018 = p.loc[(p["yearID"] >= 2016) & (p["yearID"] <= 2018) & (p["lgID"] == "NL")]

In [54]:
avg_era = p_2016_2018["ERA"].mean()
avg_pitchers = p_2016_2018.loc[(p_2016_2018["ERA"] <= avg_era + 0.5) &
                               (p_2016_2018["ERA"] >= avg_era - 0.5)].copy()

In [55]:
from pybaseball import playerid_reverse_lookup

In [58]:
avg_lookup = playerid_reverse_lookup(avg_pitchers["playerID"], key_type="bbref")

Gathering player lookup table. This may take a moment.


In [65]:
avg_pitchers = pd.merge(avg_pitchers, avg_lookup[['key_bbref', 'key_mlbam']],
         left_on="playerID", right_on='key_bbref', how='left')

In [76]:
apitchers_pitches = pitches_2016_2018.loc[(pitches_2016_2018["pitcher"].isin(avg_pitchers["key_mlbam"]))].copy()

In [77]:
#apitchers_pitches.to_csv("apitchers_pitches.csv")

In [82]:
apitchers_pitches["game_year"] = apitchers_pitches["game_date"].apply(lambda x: x.year)

In [104]:
avg_pitches_per_year = apitchers_pitches.groupby(["game_year", "pitcher"]).size().mean()
avg_pitches_per_year

1105.1290322580646

In [108]:
# pitched in 2016, 2017, and 2018

apitchers_pitches.groupby(["pitcher"]).filter(lambda x: x["game_year"].nunique() == 3 and
                                             x.size >= 3*avg_pitches_per_year).groupby(["pitcher"]).size()

pitcher
424144.0    1840
434538.0    6877
435221.0    3384
468504.0    8541
501822.0    2427
543339.0    2953
543475.0    3618
543901.0     668
571578.0    8765
592662.0    8300
592779.0    1405
595918.0    2488
596133.0    4114
605151.0    5075
605538.0    3601
606131.0    5760
606983.0     912
608379.0    6433
608566.0    6204
612672.0    4635
621107.0    4052
622097.0    2008
623184.0    2408
625643.0    4702
640463.0    1750
dtype: int64